# Deploy the bot in test mode

Run this notebook on the serverless cluster.

Select Serverless environment v3, from the settings button on the right side menu.

![Serverless env](https://github.com/brickops/databricks-botops-course/blob/main/images/serverless_env.png).

## Pre-deployment agent validation
Before registering and deploying the agent, perform pre-deployment checks using the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See Databricks documentation ([AWS](https://docs.databricks.com/en/machine-learning/model-serving/model-serving-debug.html#validate-inputs) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/model-serving-debug#before-model-deployment-validation-checks)).

#### Common setup

You can ignore the message `Core Python package version(s) changed`

In [0]:
%run ../../../libs/botops/buildsetup_serverless

#### Set the bot name

Set bot name to the name of the current folder

In [0]:
BOT_NAME = folder()


#### Task: Set the correct tripbot run id

Replace run id with the id for the run where you logged the model

In [0]:
# TRIPBOT_RUN_ID = "123"

Run a single predict to check if model can run

In [0]:
%sh pip list | grep mlflow

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{TRIPBOT_RUN_ID}/{BOT_NAME}",
    input_data={"messages": [{"role": "user", "content": "Hello!"}]},
    env_manager="uv",
)

In [0]:
# A borough question
mlflow.models.predict(
    model_uri=f"runs:/{TRIPBOT_RUN_ID}/{BOT_NAME}",
    input_data={"messages": [{"role": "user", "content": "Which borough has most traffic?"}]},
    env_manager="uv",
)

## Register the model to Unity Catalog

Before you deploy the agent, you must register the agent to Unity Catalog.

Unity catalog is Databricks universal resource registry, somewhat similar to a Data Warehouse Schema.

We must the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
# Name functions enables automatic env+user specific database naming
from brickops.datamesh.naming import catname_from_path
from brickops.datamesh.naming import dbname

In [0]:
cat = catname_from_path()
db = dbname(db="taxinycbots", cat=cat)
print("New db name: " + db)
spark.sql(f"USE catalog {cat}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")

#### Study the brickops configuration

The function `dbname()` creates the dbname according to the naming configuration.
You can find it in the repo root in `.brickopscfg/config.yml`.
It defines how naming is derived from the folder position and environment.

#### Task: Why is model path not `transport.taxinycbots.tripbot`?

Why should we deploy the test model to another path than `transport.taxinycbots.tripbot`?
Reply in the field below.

#### Register the model

Register the model un Unity catalog

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
# catalog = "training"
# schema = "testagents"
UC_MODEL_NAME = f"{db}.{BOT_NAME}"

model_uri = f"runs:/{TRIPBOT_RUN_ID}/{BOT_NAME}"
# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=model_uri, 
    name=UC_MODEL_NAME,
    env_pack="databricks_model_serving",  # faster deployment
)

In [0]:
%sh pip list | grep mlflow

In [0]:
# Show the model info
uc_registered_model_info

In [0]:
# Print current version
uc_registered_model_info.version

## Deploy the model

It can take up to 15m for the model to go live.

You can ignore the following error:

```
409 Client Error: Conflict
{"error_code":"ALREADY_EXISTS","message":"Error [f126207b-45d1-4b65-9806-1dafbf7eaa6e]: 
A monitor for experiment d77f45d... already exists. 
Set the experiment_id parameter to create a monitor for a different experiment.",
"details":[{"@type":"type.googleapis.com/google.rpc.RequestInfo",
"request_id":"f126207b-45d1-4b65-9806-1dafbf7eaa6e","serving_data":""}]}. 
```

The existing monitor will be reused.

In [0]:
from databricks import agents
agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    tags = {"endpointSource": "docs"},
    environment_vars=BOTOPS_ENV_VARS,
    scale_to_zero_enabled=True,
)

## Task: Explore the View Status, Review app and Monitor links above

What is the purpose of each of these? Answer below.

## Next steps

After your agent is deployed, you could chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See docs ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)) for details

## Task: Query your deployed endpoint

Go to the `View status` link, e.g. [https://dbc-639f4875-165d.cloud.databricks.com/ml/endpoints/agents_transport-test_paldevibe_featgh1030assis_425a3d3a_taxiny](https://dbc-639f4875-165d.cloud.databricks.com/ml/endpoints/agents_transport-test_paldevibe_featgh1030assis_425a3d3a_taxiny), and press Use array -> Query, and as request use:

```
{"messages": [{"role": "user", "content": "Which borough has most traffic?"}]}
```

![query_endpoint](https://github.com/brickops/databricks-botops-course/blob/main/images/query_endpoint.png)

## Task: Chat with the deployed test bot

...and figure out where the feedback ends up. You can find an already working [review bot app here](https://dbc-639f4875-165d.cloud.databricks.com/ml/review-v2/chat?endpoint=agents_transport-test_paldevibe_featgh1030assis_425a3d3a_taxiny&o=3132861941250972).

The first message you should send is just "Hello". Note the Feedback functionality in the review app, like thumbs up and thumbs down.

In some versions of Agents deployment you might get the Error "Received an invalid and unexpected value from the API" when asking about the taxi data. We are working on fixing this problem, which has occurred in a newer version of the agents deployment service.